# Generate Map

> Class that generates the data map

In [ ]:
#| default_exp app

In [ ]:
#| export
from nbdev.showdoc import *
# TODO work out how to get around below hack
try:
    from SolomonIslandsDataMap.load_data import *
except: 
    from load_data import *
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio # Unless this is used graphs will not be dynamic?
import json
import pandas as pd
import numpy as np
from fastcore.test import *
from dash import Dash, dcc, Output, Input, html  # pip install dash
import dash_bootstrap_components as dbc    # pip install dash-bootstrap-components
import random

## Load Data

In [ ]:
#| export
sol_geo = SolomonGeo.read_test()
geo_df = sol_geo.geo_df

## Setup Dash Components

In [ ]:
#| export
# Build your components
# FYI the best themes seem to be: [Darkly, Flatly, Minty, Slate]
app = Dash(__name__, external_stylesheets=[dbc.themes.MINTY])
server = app.server


mytitle = dcc.Markdown(children='')
mygraph = dcc.Graph(figure={})
aggs = geo_df.loc[:, 'agg'].unique()
dropdown = dcc.Dropdown(options=aggs,
                        value=aggs[0],  # initial value displayed when page first loads
                        clearable=False)

## Create a sidebar
The sidebar will 

In [ ]:
#| export
# Note, for now I am not using a sidebar style as I do not want to fix the width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "24rem",
    "padding": "2rem 1rem",
    #"background-color": "#f8f9fa",
}


sidebar = html.Div(
    [
        html.H2("Filters"),
        html.Hr(),
        dbc.Nav(
            [
                html.P("Data Aggregation"),
                dropdown,
                html.Br(),
                #dcc.Dropdown(id = 'two'),
                #html.Br(),
                #dcc.Dropdown(id = 'three')

            ],
            vertical=True,
            pills=True,
        ),
    ],
    #style=SIDEBAR_STYLE,
)



## Customize the Layout

In [ ]:

app.layout = dbc.Container([
                dbc.Row([
                    dbc.Col(),

                    dbc.Col([mytitle],width = 9, style = {'margin-left':'7px','margin-top':'7px'})
                    ]),
                dbc.Row(
                    [dbc.Col(sidebar, width = 3),
                    dbc.Col([mygraph], width = 9)#, style = {'margin-left':'15px', 'margin-top':'7px', 'margin-right':'15px'})
                     ], justify = 'center'),                    
                ], fluid = True)

## Build callbacks to create interaction

In [ ]:
#| export
# Callback allows components to interact
@app.callback(
    Output(mygraph, 'figure'),
    Output(mytitle, 'children'),
    Input(dropdown, 'value')
)
def update_graph(user_input):  # function arguments come from the component property of the Input
    fig = go.Figure(go.Choroplethmapbox(
                            geojson=sol_geo.get_geojson(agg_filter = user_input),
                           locations=sol_geo.get_df(agg_filter = user_input).index,
                           z = sol_geo.get_df(agg_filter = user_input)['total_pop'],
                           colorscale="deep",
                            marker_line_width = 0,
                            zauto=True))

    fig.update_layout(mapbox_style = 'carto-positron',
                        mapbox_zoom = 5,
                        mapbox_center={"lat": -9.565766, "lon": 162.012453},
    )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    # returned objects are assigned to the component property of the Output
    return fig, '# Solomon Islands Data map - ' + user_input


## Run the dash app

In [ ]:
#| export
#| server
# Run app
if __name__=='__main__':
    app.run_server(debug=True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()